## Running Summary

### Load Env

In [1]:
%load_ext dotenv
%dotenv

### Create State

In [2]:
from langgraph.graph import MessagesState

In [3]:
class State(MessagesState):
    summary: str

In [13]:
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

In [8]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    temperature=0.0,
    model="gpt-4o-mini",
)

In [9]:
def summarize_conversation(state: State) -> State:
    summary = state.get("summary")

    if summary:
        sys_msg = (
            f"This is a summary of the conversation to date: {summary}"
            "\n\n Extend the summary by taking into account the latest messages given to you above."
        )
    else:
        sys_msg = "Create a summary of the conversation above."
    
    message = state["messages"] + [HumanMessage(content=sys_msg)]
    response = llm.invoke(message)

    delete_messages = [RemoveMessage(message.id) for message in state["messages"][:-2]]
    return {"messages": delete_messages, "summary": response.content}


In [10]:
def model_invocation(state: State):
    summary = state.get("summary")
    if summary:
        sys_msg = f"Summary of previous conversation: {summary}"
        messages = [SystemMessage(sys_msg)] + state["messages"]
    else:
        messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}

In [12]:
def should_continue(state: State):
    """Return the next node to execute"""
    messages = state.get("messages")
    if len(messages) > 6:
        return "summarize_conversation"
    return END

### Build A Graph

In [14]:
builder = StateGraph(State)

builder.add_node("conversation", model_invocation)
builder.add_node("summarize_conversation", summarize_conversation)

builder.add_edge(START, "conversation")
builder.add_conditional_edges("conversation", should_continue)
builder.add_edge("summarize_conversation", END)

memory = MemorySaver()

graph = builder.compile()